In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
import os
from typing import cast
import xlsxwriter as pw
from xlsxwriter.chart import Chart      # ← add this
import polars as pl
import re

In [4]:
path = r"C:\Users\irfan\OneDrive\Desktop\Python Projects\Small_Cleaning_Task\Dirty_csv_Dataset.csv"

# Load the CSV directly into a DataFrame
df = pd.read_csv(path)


In [5]:
# convert pandas df to polars, clean, then convert back
pl_df = pl.from_pandas(df)

In [6]:
cleaned = (
    pl_df
    .rename(
        {
            col: re.sub(r'\s+', '_', col.strip().replace('-', '').lower())
            for col in pl_df.columns
        }
    )  # rename columns: strip whitespace, remove hyphens, lowercase and convert spaces to underscores
    .with_columns([
        pl.col("first_name")
          .str.to_lowercase()
          .str.to_titlecase()
          .alias("first_name"),
        pl.col("last_name")
          .str.to_lowercase()
          .str.to_titlecase()
          .alias("last_name")
    ])  # normalize first and last names to Title Case
    .filter(
        (pl.col("first_name") != "") &
        (pl.col("last_name")  != "")
    )  # drop rows with empty first or last name
    .with_columns(
        pl.col("email")
          .str.split(r";")
          .alias("email")
    )  # split multiple emails into a list
    .explode("email")  # expand each email entry into its own row
    .filter(
        pl.col("email")
          .str.contains(r"\.com")
    )  # keep only email addresses ending with .com

)

In [7]:
df = cleaned.to_pandas()

In [8]:
print(df)

  first_name last_name                      email    signup_date amount_paid  \
0      Alice     Smith    alice.smith@example.com     2025/01/15   $1,200.00   
1      Alice     Smith           asmith@gmail.com     2025/01/15   $1,200.00   
2    Charlie    O'Neil  charlie.oneil@example.com     2025.03.01       950.5   
3      David     Brown    david.brown@example.com  April 5, 2025    1,100.00   
4      David     Brown          d.brown@yahoo.com  April 5, 2025    1,100.00   

     status  
0    Active  
1    Active  
2  Inactive  
3    ACTIVE  
4    ACTIVE  


In [9]:
from datetime import datetime

def parse_signup_date(x):
    for fmt in ("%Y-%m-%d", "%Y.%m.%d", "%B %d, %Y"):
        try:
            return datetime.strptime(x, fmt)
        except (ValueError, TypeError):
            continue
    # fallback to pandas/dateutil for anything else
    return pd.to_datetime(x, errors="coerce")

# ...existing code...
df["signup_date"] = df["signup_date"].apply(parse_signup_date)
# # drop any that still failed to parse
# df = df[df["signup_date"].notna()]
# finally format uniformly
df["signup_date"] = df["signup_date"].dt.strftime("%Y-%m-%d")
# ...existing code...

In [10]:
display(df)

,first_name,last_name,email,signup_date,amount_paid,status
0,Alice,Smith,alice.smith@example.com,2025-01-15,"$1,200.00",Active
1,Alice,Smith,asmith@gmail.com,2025-01-15,"$1,200.00",Active
2,Charlie,O'Neil,charlie.oneil@example.com,2025-03-01,950.5,Inactive
3,David,Brown,david.brown@example.com,2025-04-05,"1,100.00",ACTIVE
4,David,Brown,d.brown@yahoo.com,2025-04-05,"1,100.00",ACTIVE


In [11]:
# remove currency symbols and thousand separators, convert to float
df["amount_paid"] = (
    df["amount_paid"]
    .str.replace(r"[\$,]", "", regex=True)
    .astype(float)
)
df.head()

,first_name,last_name,email,signup_date,amount_paid,status
0,Alice,Smith,alice.smith@example.com,2025-01-15,1200.0,Active
1,Alice,Smith,asmith@gmail.com,2025-01-15,1200.0,Active
2,Charlie,O'Neil,charlie.oneil@example.com,2025-03-01,950.5,Inactive
3,David,Brown,david.brown@example.com,2025-04-05,1100.0,ACTIVE
4,David,Brown,d.brown@yahoo.com,2025-04-05,1100.0,ACTIVE


In [12]:
# trim any leading/trailing whitespace and convert to lowercase
df['status'] = df['status'].str.strip().str.lower()

# verify the change
df.head()

,first_name,last_name,email,signup_date,amount_paid,status
0,Alice,Smith,alice.smith@example.com,2025-01-15,1200.0,active
1,Alice,Smith,asmith@gmail.com,2025-01-15,1200.0,active
2,Charlie,O'Neil,charlie.oneil@example.com,2025-03-01,950.5,inactive
3,David,Brown,david.brown@example.com,2025-04-05,1100.0,active
4,David,Brown,d.brown@yahoo.com,2025-04-05,1100.0,active


In [13]:
import os

output_path = os.path.join(os.getcwd(), "cleaned_data.xlsx")
df.to_excel(output_path, index=False)
os.startfile(output_path)